# Pv4 Data Access

This page provides information about how to access data from [Plasmodium vivax version 4 (Pv4)](https://www.malariagen.net/resource/30) project via Google Cloud. This includes sample metadata and single nucleotide polymorphism (SNP) calls for 1,895 samples from 27 countries. This release spans [11 partner studies and 3 external studies](https://www.malariagen.net/sites/default/files/Pv4%20contributing%20partner%20studies.pdf).

This notebook illustrates how to read data directly from the cloud, **without having to first download any data locally**. This notebook can be run from any computer, but will work best when run from a compute node within Google Cloud, because it will be physically closer to the data and so data transfer is faster. For example, this notebook can be run via MyBinder or Google Colab which are free interactive computing service running in the cloud.

To launch this notebook in the cloud and run it for yourself, click the launch icon () at the top of the page and select one of the cloud computing services available.

## Setup

Running this notebook requires some Python packages to be installed. These packages can be installed via pip or conda. E.g.:

In [3]:
# The pv4 module is not yet included in the malariagen package. When it is the below should work.
# !pip install -q malariagen_data

To make accessing these data more convenient, we’ve created the malariagen_data Python package, which is available from PyPI. This is experimental so please let us know if you find any bugs or have any suggestions.

Now import the packages we’ll need to use here.

In [1]:
from malariagen_data.pv4 import Pv4
# import malariagen_data (above line will eventually be substituted for this)

In [2]:
import numpy as np
import dask
import dask.array as da
from dask.diagnostics.progress import ProgressBar
# silence some warnings
dask.config.set(**{'array.slicing.split_large_chunks': False})
import allel

To access the Pv4 data stored on google cloud use the following code:

In [3]:
pv4 = Pv4("gs://pv4_staging/")

## Metadata

Data on the samples that were sequenced as part of this resource are available. It includes the time and place of collection, quality metrics, and accesion numbers.

To see all the information available, load sample metadata into a pandas dataframe:

In [4]:
pv4_metadata = pv4.sample_metadata()

pv4_metadata.head()

,Sample,Study,Site,First-level administrative division,Country,Lat,Long,Year,ENA,All samples same individual,Population,% callable,QC pass,Exclusion reason,Is returning traveller
0,BBH-1-125,X0009-PV-ET-LO,Jimma,Ethiopia: Oromia,Ethiopia,7.683331,36.851318,2016,ERR2678989,BBH-1-125,AF,88.52,True,Analysis_set,False
1,BBH_1_132,X0009-PV-ET-LO,Jimma,Ethiopia: Oromia,Ethiopia,7.683331,36.851318,2016,ERR2678991,BBH_1_132,AF,90.20,True,Analysis_set,False
2,BBH_1_137,X0009-PV-ET-LO,Jimma,Ethiopia: Oromia,Ethiopia,7.683331,36.851318,2016,ERR2679003,BBH_1_137,AF,87.09,True,Analysis_set,False
3,BBH_1_153,X0009-PV-ET-LO,Jimma,Ethiopia: Oromia,Ethiopia,7.683331,36.851318,2016,ERR2678992,BBH_1_153,AF,90.60,True,Analysis_set,False
4,BBH_1_162,X0009-PV-ET-LO,Jimma,Ethiopia: Oromia,Ethiopia,7.683331,36.851318,2016,ERR2678993,BBH_1_162,AF,91.67,True,Analysis_set,False


In [5]:
print("The dataset has {} samples and {} fields".format(pv4_metadata.shape[0],pv4_metadata.shape[1]))

The dataset has 1895 samples and 15 fields


We can explore each of the fields: 

- The <font color='purple'>Sample</font> column gives the unique sample identifier used throughout all Pv4 analyses.


- The <font color='purple'>Study</font> refers to the partner study which collected the sample.


- The <font color='purple'>Country</font>, <font color='purple'>First-level administrative division</font> and <font color='purple'>Site</font> describe the location where the sample was collected from. 


- The <font color='purple'>Lat</font> & <font color='purple'>Long</font> contain the GADM coordinates for each country.


- The <font color='purple'>Year</font> column gives the time of sample collection.


- The <font color='purple'>ENA</font> column gives the run accession(s) for the sequencing read data for each sample.


- The <font color='purple'>All samples same individual</font> column identifies samples set collected from the same individual. 
    
    
- The <font color='purple'>Population</font> column gives the population to which the sample has been assigned to, based on the degree of genetic similarity to other samples. The possible values are: Africa-Central (AF-C), Africa - East (AF-E), Africa - Northeast (AF-NE), Africa - West (AF-W), Asia - South Asia - East (AS-SA-E), Asia - South Asia - West (AS-SA-W), Asia - Southeast Asia - East (AS-SEA-E), Asia - Southeast Asia - West (AS-SEA-W), Oceania - New Guinea (OC-NG), South America (SA). 


- The <font color='purple'>% callable</font> column refers to the %  of the genome with coverage of at least 5 reads and less than 10% of reads with mapping quality 0. 
 
 
- The <font color='purple'>QC pass</font> column defines whether the sample passed (True) or failed (False) QC. 
    
    
- The <font color='purple'>Exclusion reason</font> describes the reason why the particular sample was excluded from the main analysis.

        
- The <font color='purple'>Is returning traveller</font> column states if the sample was collected from a person returning from travel. The region assigned in this case was based on the reported country of travel.

The python package [Pandas](https://pandas.pydata.org/) can be used to explore and query the sample metadata in different ways. For example, here is a summary of the numbers of samples grouped by the country they were collected in:

In [6]:
pv4_metadata.groupby("Country").size()

Country
Afghanistan         250
Bangladesh           28
Bhutan                9
Brazil               71
Cambodia            236
China                 5
Colombia            112
El Salvador           2
Ethiopia            203
India                14
Indonesia           282
Iran                 15
Madagascar            4
Malaysia            109
Mauritania            1
Mexico               20
Myanmar               9
Nicaragua             1
North Korea           1
Panama                1
Papua New Guinea     47
Peru                123
Philippines           6
Sri Lanka             2
Sudan                13
Thailand            192
Vietnam             139
dtype: int64

## Variant Calls 

These files contain details of 4,571,056 discovered variant genome positions. These variants were discovered amongst all samples from the release.

945,649 of these variant positions are SNPs, with the remainder being either short insertion/deletions (indels), or a combination of SNPs and indels. 

Data on variant calls, including the genomic positions, alleles, and genotypes, can be accessed as an xarray Dataset:

In [7]:
variant_dataset = pv4.variant_calls()
variant_dataset

<xarray.Dataset>
Dimensions:              (variants: 4571056, alleles: 7, samples: 1895, ploidy: 2)
Coordinates:
    variant_position     (variants) int32 dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_chrom        (variants) object dask.array<chunksize=(65536,), meta=np.ndarray>
    sample_id            (samples) object dask.array<chunksize=(1895,), meta=np.ndarray>
Dimensions without coordinates: variants, alleles, samples, ploidy
Data variables:
    variant_allele       (variants, alleles) object dask.array<chunksize=(65536, 1), meta=np.ndarray>
    variant_filter_pass  (variants) bool dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_is_snp       (variants) bool dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_numalt       (variants) int32 dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_CDS          (variants) bool dask.array<chunksize=(65536,), meta=np.ndarray>
    call_genotype        (variants, samples, ploidy) int8 dask.array<chunksize=(65536, 64, 2), meta=np.ndarray>
    call_AD              (variants, samples, alleles) int16 dask.array<chunksize=(65536, 64, 7), meta=np.ndarray>

The default returns a basic set of data most commonly used for data analysis. However, for more complex analysis the full range of variables available in the zarr can be accessed by setting the extended flag to `True`, as shown below:

In [8]:
extended_variant_dataset = pv4.variant_calls(extended=True)
extended_variant_dataset

<xarray.Dataset>
Dimensions:                                   (variants: 4571056, alleles: 7, samples: 1895, ploidy: 2, genotypes: 3, alt_alleles: 6)
Coordinates:
    variant_position                          (variants) int32 dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_chrom                             (variants) object dask.array<chunksize=(65536,), meta=np.ndarray>
    sample_id                                 (samples) object dask.array<chunksize=(1895,), meta=np.ndarray>
Dimensions without coordinates: variants, alleles, samples, ploidy, genotypes, alt_alleles
Data variables: (12/42)
    variant_allele                            (variants, alleles) object dask.array<chunksize=(65536, 1), meta=np.ndarray>
    variant_filter_pass                       (variants) bool dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_is_snp                            (variants) bool dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_numalt                            (variants) int32 dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_CDS                               (variants) bool dask.array<chunksize=(65536,), meta=np.ndarray>
    call_genotype                             (variants, samples, ploidy) int8 dask.array<chunksize=(65536, 64, 2), meta=np.ndarray>
    ...                                        ...
    variant_SNPEFF_IMPACT                     (variants) object dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_SNPEFF_TRANSCRIPT_ID              (variants) object dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_SOR                               (variants) float32 dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_VQSLOD                            (variants) float32 dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_VariantType                       (variants) object dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_altlen                            (variants, alt_alleles) int32 dask.array<chunksize=(65536, 6), meta=np.ndarray>

Each of the elements in this xarray dataset is a [dask array](https://docs.dask.org/en/latest/array.html). The individual dask arrays can be accessed as follows, replacing the string with the variable you are looking for:

In [9]:
pos = variant_dataset["variant_position"].data
pos

dask.array<array, shape=(4571056,), dtype=int32, chunksize=(65536,), chunktype=numpy.ndarray>

## Genotypes

Genotypes for individual samples are available.

Genotypes are stored as a three-dimensional array, where:

* the first dimension corresponds to genomic positions,
* the second dimension is samples,
* the third dimension is ploidy (2).

Values coded as integers, where -1 represents a missing value, 0 represents the reference allele, and 1, 2, and 3 represent alternate alleles.

Variant genotypes can be accessed as dask arrays as shown below.

In [10]:
gt = variant_dataset["call_genotype"].data
gt

,Array,Chunk
Bytes,16.13 GiB,8.00 MiB
Shape,"(4571056, 1895, 2)","(65536, 64, 2)"
Count,2100 Tasks,2100 Chunks
Type,int8,numpy.ndarray


Note that the columns of this array (second dimension) match the rows in the sample metadata. You can use this correspondance to apply further subsetting operations to the genotypes by querying the sample metadata. E.g.:

In [11]:
loc_cambodia = pv4_metadata.eval("Country == 'Cambodia'").values
print(f"found {np.count_nonzero(loc_cambodia)} samples from Cambodia")
variant_dataset_cambodia = variant_dataset.isel(samples=loc_cambodia)
variant_dataset_cambodia

found 236 samples from Cambodia


<xarray.Dataset>
Dimensions:              (variants: 4571056, alleles: 7, samples: 236, ploidy: 2)
Coordinates:
    variant_position     (variants) int32 dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_chrom        (variants) object dask.array<chunksize=(65536,), meta=np.ndarray>
    sample_id            (samples) object dask.array<chunksize=(236,), meta=np.ndarray>
Dimensions without coordinates: variants, alleles, samples, ploidy
Data variables:
    variant_allele       (variants, alleles) object dask.array<chunksize=(65536, 1), meta=np.ndarray>
    variant_filter_pass  (variants) bool dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_is_snp       (variants) bool dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_numalt       (variants) int32 dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_CDS          (variants) bool dask.array<chunksize=(65536,), meta=np.ndarray>
    call_genotype        (variants, samples, ploidy) int8 dask.array<chunksize=(65536, 2, 2), meta=np.ndarray>
    call_AD              (variants, samples, alleles) int16 dask.array<chunksize=(65536, 2, 7), meta=np.ndarray>

The data on genomic variants can be loaded into memory as [numpy](https://numpy.org/doc/stable/reference/generated/numpy.array.html) arrays as shown in the following example, where we read genotypes for the first 5 SNPs and the first 3 samples:


In [12]:
g = gt[:5, :3, :].compute()
g

array([[[-1, -1],
        [ 0,  0],
        [-1, -1]],

       [[-1, -1],
        [ 0,  0],
        [-1, -1]],

       [[-1, -1],
        [ 0,  0],
        [-1, -1]],

       [[-1, -1],
        [ 0,  0],
        [-1, -1]],

       [[-1, -1],
        [ 0,  0],
        [-1, -1]]], dtype=int8)

If you want to work with the genotype calls, you may find it convenient to use [scikit-allel](https://scikit-allel.readthedocs.io/en/stable/). E.g., the code below sets up a genotype array using the Cambodian samples subset we created above.

In [13]:
# use the scikit-allel wrapper class for genotype calls
gt = allel.GenotypeDaskArray(variant_dataset_cambodia["call_genotype"].data)
gt

<GenotypeDaskArray shape=(4571056, 236, 2) dtype=int8>

## Genome Annotations 

Gene annotations provide information on which regions of the genome contain DNA sequences that encode genes, which are transcribed and spliced into messenger RNA (mRNA) and then translated to make proteins.

For convenience, we’ve added some functionality to the malariagen_data package for loading these gene annotations into a pandas data frame as shown below:

In [14]:
genome_features = pv4.genome_features()
genome_features

,contig,source,type,start,end,score,strand,phase,ID,Parent,Name,alias
0,PvP01_00_v1,chado,contig,1,402823,NaN,+,NaN,Transfer.PvP01_00_1.final,NaN,NaN,NaN
1,PvP01_00_v1,chado,gene,53,669,NaN,+,NaN,PVP01_0000010,NaN,NaN,NaN
2,PvP01_00_v1,chado,mRNA,53,669,NaN,+,NaN,PVP01_0000010.1,PVP01_0000010,NaN,NaN
3,PvP01_00_v1,chado,CDS,53,135,NaN,+,0.0,PVP01_0000010.1:exon:1,PVP01_0000010.1,NaN,NaN
4,PvP01_00_v1,chado,CDS,199,415,NaN,+,1.0,PVP01_0000010.1:exon:2,PVP01_0000010.1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
38676,PvP01_MIT_v1,chado,CDS,4782,5912,NaN,+,0.0,PVP01_MIT03400.1:exon:1,PVP01_MIT03400.1,NaN,NaN
38677,PvP01_MIT_v1,chado,polypeptide,4782,5912,NaN,+,NaN,PVP01_MIT03400.1:pep,NaN,NaN,NaN
38678,PvP01_MIT_v1,chado,gene,5914,5985,NaN,+,NaN,PVP01_MIT03500,NaN,NaN,NaN
38679,PvP01_MIT_v1,chado,rRNA,5914,5985,NaN,+,NaN,PVP01_MIT03500.1,PVP01_MIT03500,NaN,NaN


The above loads a default set of attributes `"ID", "Parent", "Name", "alias"`. To access all features set `attributes` to `"*"`.

In [15]:
pv4.genome_features(attributes="*")

,contig,source,type,start,end,score,strand,phase,Dbxref,Derives_from,...,non_cytoplasmic_polypeptide_region,orthologous_to,polypeptide_domain,previous_systematic_id,product,signal_peptide,stop_codon_redefined_as_selenocysteine,synonym,translation,transmembrane_polypeptide_region
0,PvP01_00_v1,chado,contig,1,402823,NaN,+,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PvP01_00_v1,chado,gene,53,669,NaN,+,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PvP01_00_v1,chado,mRNA,53,669,NaN,+,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PvP01_00_v1,chado,CDS,53,135,NaN,+,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PvP01_00_v1,chado,CDS,199,415,NaN,+,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38676,PvP01_MIT_v1,chado,CDS,4782,5912,NaN,+,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38677,PvP01_MIT_v1,chado,polypeptide,4782,5912,NaN,+,NaN,NaN,PVP01_MIT03400.1,...,";query 47-73;,;query 126-134;,;query 192-218;,...",Pknowlesi:PKNH_MIT01900 link=PKNH_MIT01900.1:p...,"iprscan;InterPro:IPR036150 :\tCytochrome b/b6,...",NaN,term=cytochrome b;,NaN,NaN,NaN,mnyysinlakahllnypcplninflwnygfllgiiffiqiltgvfl...,";query 24-46;,;query 74-96;,;query 103-125;,;q..."
38678,PvP01_MIT_v1,chado,gene,5914,5985,NaN,+,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38679,PvP01_MIT_v1,chado,rRNA,5914,5985,NaN,+,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Or to get a specific set of attributes specify them in a list

In [16]:
pv4.genome_features(attributes=['alias','comment','product'])

,contig,source,type,start,end,score,strand,phase,alias,comment,product
0,PvP01_00_v1,chado,contig,1,402823,NaN,+,NaN,NaN,Archived from fasta_record feature Transfer.Pv...,NaN
1,PvP01_00_v1,chado,gene,53,669,NaN,+,NaN,NaN,NaN,NaN
2,PvP01_00_v1,chado,mRNA,53,669,NaN,+,NaN,NaN,NaN,NaN
3,PvP01_00_v1,chado,CDS,53,135,NaN,+,0.0,NaN,NaN,NaN
4,PvP01_00_v1,chado,CDS,199,415,NaN,+,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
38676,PvP01_MIT_v1,chado,CDS,4782,5912,NaN,+,0.0,NaN,NaN,NaN
38677,PvP01_MIT_v1,chado,polypeptide,4782,5912,NaN,+,NaN,NaN,NaN,term=cytochrome b;
38678,PvP01_MIT_v1,chado,gene,5914,5985,NaN,+,NaN,NaN,NaN,NaN
38679,PvP01_MIT_v1,chado,rRNA,5914,5985,NaN,+,NaN,NaN,NaN,NaN


## Genome Reference 

We mapped sequence reads for all samples against the PvP01 reference genome.

For convenience, the reference genome sequence can be loaded as a dask array, e.g.:

In [17]:
ref = pv4.genome_sequence()
ref 

dask.array<concatenate, shape=(24250245,), dtype=|S1, chunksize=(440322,), chunktype=numpy.ndarray>

This can be loaded as a numpy array using the following 

In [18]:
ref.compute()

array([b'a', b'a', b't', ..., b't', b'a', b'a'], dtype='|S1')

The reference can also be subset by contig. 

The set of contigs used can be accessed as follows:

In [19]:
pv4.contigs

['PvP01_01_v1',
 'PvP01_02_v1',
 'PvP01_03_v1',
 'PvP01_04_v1',
 'PvP01_05_v1',
 'PvP01_06_v1',
 'PvP01_07_v1',
 'PvP01_08_v1',
 'PvP01_09_v1',
 'PvP01_10_v1',
 'PvP01_11_v1',
 'PvP01_12_v1',
 'PvP01_13_v1',
 'PvP01_14_v1',
 'PvP01_API_v1',
 'PvP01_MIT_v1']

To load a single contig

In [20]:
pv4.genome_sequence(region='PvP01_01_v1')

dask.array<array, shape=(1021664,), dtype=|S1, chunksize=(255416,), chunktype=numpy.ndarray>

To load multiple contigs specify them in a list. The data will be concatenated.

In [21]:
pv4.genome_sequence(region=['PvP01_01_v1','PvP01_06_v1','PvP01_10_v1'])

dask.array<concatenate, shape=(3613299,), dtype=|S1, chunksize=(387211,), chunktype=numpy.ndarray>

You can also specify a specific region of the contig.

In [22]:
pv4.genome_sequence(region=['PvP01_01_v1','PvP01_06_v1:15-20','PvP01_10_v1:40-50'])

dask.array<concatenate, shape=(1021681,), dtype=|S1, chunksize=(255416,), chunktype=numpy.ndarray>

If you know the gene name you would like to access, but aren't sure what the ID would be you can access this through the annotations. Below is an example for `?`.  

In [26]:
gene_name = str(genome_features.loc[genome_features.Name == 'ORC2'].ID.values)
print(gene_name)

['PVP01_0105500']


You can then enter this as the region

In [27]:
pv4.genome_sequence(region='PVP01_0105500')

dask.array<getitem, shape=(2625,), dtype=|S1, chunksize=(2625,), chunktype=numpy.ndarray>